In [173]:
import cv2
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from skimage.feature import greycomatrix,greycoprops

class Key_Frame_Extraction:
    def __init__(self,offset):
        self.numCluster = 1
        self.Cluster = []
        self.centroid = []
        self.a = offset
    
    def get_GLCM_Feature(self,g):
        gl = np.array([
        greycoprops(g, 'contrast'),
        greycoprops(g, 'homogeneity'),
        greycoprops(g, 'energy'),
        greycoprops(g, 'correlation')])
        return gl
    
    def get_Frame(self,video_path):
        vidcap = cv2.VideoCapture(video_path)
        test_image_set = []
        count = 0
        frame_count = 0
        while(vidcap.isOpened):
            ret, image = vidcap.read()
            #print(ret)
            if ret:
                count +=1
                if count%10 == 0:
                    frame_count+=1
                    test_image_set.append(cv2.cvtColor(image,cv2.COLOR_RGB2HSV))
                    #print(f"frame {frame_count}")
            else:
                    break
        vidcap.release()
        return test_image_set
        
    def CombineSimilarity(self,img,centroid):
        hist = cv2.calcHist([img],[0],None,[256],[0,256])
        hist = cv2.normalize(hist,hist).flatten()
        HisSimilarity = np.array([ cv2.compareHist(hist,x[1],cv2.HISTCMP_INTERSECT) for x in centroid])

        glcm= greycomatrix(cv2.cvtColor(img,cv2.COLOR_RGB2GRAY),[1],[0],symmetric = True,normed = True)
        glcm = self.get_GLCM_Feature(glcm)
        GLCMSimilarity = np.array([np.sum((glcm - x[2])**2)**0.5 for x in centroid])

        return 0.7*HisSimilarity - 0.3*GLCMSimilarity
    
    def get_KeyFrame(self,frame_set):
        train_x = frame_set
        
        self.Cluster.append([train_x[0]])
        hist = cv2.calcHist([train_x[0]],[0],None,[256],[0,256])
        g = greycomatrix(cv2.cvtColor(train_x[0],cv2.COLOR_RGB2GRAY),[1],[0],symmetric = True,normed = True)
                         
        self.centroid.append([train_x[0],cv2.normalize(hist,hist).flatten(),self.get_GLCM_Feature(g)])
        for idx,x in enumerate(train_x[1:]):
            result = self.CombineSimilarity(x,self.centroid)

            s = np.max(result)
            maxidx = np.argmax(result)
            #print(f'{s} ')
            if(s > self.a):
                #print(f'Cluster idx : {maxidx}')
                self.Cluster[maxidx].append(x)
                numFrame = len(self.Cluster[maxidx])
                hist = cv2.calcHist([self.Cluster[maxidx][numFrame//2]],[0],None,[256],[0,256])
                g = greycomatrix(cv2.cvtColor(self.Cluster[maxidx][numFrame//2],cv2.COLOR_RGB2GRAY),[1],[0],symmetric = True,normed = True)        
                self.centroid[maxidx] = [self.Cluster[maxidx][numFrame//2],cv2.normalize(hist,hist).flatten(),self.get_GLCM_Feature(g)]
            else:
                #print(f'Cluster idx : {self.numCluster}')
                self.Cluster.append([x])
                self.numCluster+=1
                hist = cv2.calcHist([x],[0],None,[256],[0,256])
                g = greycomatrix(cv2.cvtColor(x,cv2.COLOR_RGB2GRAY),[1],[0],symmetric = True,normed = True)
                self.centroid.append([x,cv2.normalize(hist,hist).flatten(),self.get_GLCM_Feature(g)])
        result= []
        for i in range(len(cl.centroid)):
            if len(cl.Cluster[i]) >= len(train_x)*0.03:
                result.append(cv2.cvtColor(cl.centroid[i][0],cv2.COLOR_HSV2RGB))
        return result


In [174]:
cl = Key_Frame_Extraction(offset = 0.7)
Frame_set = cl.get_Frame("./test_video_2.mp4")

In [175]:
len(Frame_set)

376

In [176]:
Key_Frame = cl.get_KeyFrame(Frame_set)

In [177]:
len(Key_Frame)

11

In [178]:
count = 0
for i in range(len(Key_Frame)):
    print(i)
    cv2.imwrite('test_image_' + str(count) + '.jpg',Key_Frame[i])
    count += 1

0
1
2
3
4
5
6
7
8
9
10
